# The Model - La Destapada

In [ ]:
import flopy
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the model

In [ ]:
# load simulation
sim = flopy.mf6.MFSimulation.load(sim_ws="model",
                                  verbosity_level=0)
# load flow model
gwf = sim.get_model()

# Let's check the grid

In [ ]:
gwf.dis.nlay.get_data(), gwf.dis.nrow.get_data(), gwf.dis.ncol.get_data()

In [ ]:
# plot grid
fig, ax = plt.subplots(figsize=(10,10))
mv = flopy.plot.PlotMapView(model=gwf)
mv.plot_grid(lw=0.5)
mv.plot_bc('WEL-dewater', label='Dewater Wells')
mv.plot_bc('WEL-mar', label='Injection Wells')

mv.plot_bc('DRN', color='green', label='Drain - GDE')
mv.plot_bc('GHB', color='blue', label='GHB - regional aquifer')

# mv.plot_array(gwf.dis.idomain.get_data(), color='gray', alpha=1)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
mv = flopy.plot.PlotMapView(model=gwf)
mv.plot_grid(lw=0.5)
arr = mv.plot_array(gwf.npf.k.get_data(), alpha=1)
cb = plt.colorbar(mappable=arr, ax=ax, aspect=5.1)

# Let's check the time discretization

El modelo empieza en Enero del 2025 y tiene 3 periodos de estress de 1 time step cada uno.

In [ ]:
tdis = sim.tdis

# the start date time
t0 = tdis.start_date_time.get_data()
t0

In [ ]:
# the period data
period_df = pd.DataFrame(tdis.perioddata.get_data())
period_df

In [ ]:
tot_tim = period_df.perlen.sum()
tot_tim, "days"

In [ ]:
# sim end time
tf = pd.to_datetime(t0) + pd.Timedelta(days=tot_tim)
tf

# Let's check some BCs

In [ ]:
gwf.wel[0]

In [ ]:
gwf.wel[0].stress_period_data.get_data()

for kper in gwf.wel[0].stress_period_data.get_data():
    display(pd.DataFrame(gwf.wel[0].stress_period_data.get_data()[kper]))

In [ ]:
for kper in gwf.wel[1].stress_period_data.get_data():
    display(pd.DataFrame(gwf.wel[1].stress_period_data.get_data()[kper]))

# Observations

In [ ]:
obs = pd.read_csv(os.path.join("model", 
                         'model.obs_continuous_model.obs.head.wel-mar.csv.txt'),
                         sep=" ", 
                         header=None,
                         skipinitialspace=True)

obs_i = obs[3]
obs_j = obs[4] 

# Get model dimensions
nlay = gwf.dis.nlay.get_data()
nrow = gwf.dis.nrow.get_data()
ncol = gwf.dis.ncol.get_data()

# Create empty array
arr = np.zeros((nlay, nrow, ncol))

# Mask with observation indices (assuming layer=0 for all obs)
for i, j in zip(obs_i, obs_j):
    arr[0, int(i)-1, int(j)-1] = 1 

fig, ax = plt.subplots(figsize=(10,10))
mv = flopy.plot.PlotMapView(model=gwf)
mv.plot_grid(lw=0.5)
mv.plot_array(arr[0],  alpha=1)
ax.set_title('Observation Points in Injection Wells')
# Plot observation points
plt.show()

In [ ]:
obs = pd.read_csv(os.path.join("model", 
                         'model.obs_continuous_model.obs.head.pit.csv.txt'),
                         sep=" ", 
                         header=None,
                         skipinitialspace=True)

obs_i = obs[3]
obs_j = obs[4] 

# Get model dimensions
nlay = gwf.dis.nlay.get_data()
nrow = gwf.dis.nrow.get_data()
ncol = gwf.dis.ncol.get_data()

# Create empty array
arr = np.zeros((nlay, nrow, ncol))

# Mask with observation indices (assuming layer=0 for all obs)
for i, j in zip(obs_i, obs_j):
    arr[0, int(i)-1, int(j)-1] = 1 

fig, ax = plt.subplots(figsize=(10,10))
mv = flopy.plot.PlotMapView(model=gwf)
mv.plot_grid(lw=0.5)
mv.plot_array(arr[0],  alpha=1)
ax.set_title('Observation Points inside Pit')
# Plot observation points
plt.show()